In [ ]:
from pynq import Overlay
import time

# 1. Load Bitstream
overlay = Overlay("project.bit")
gpio_in  = overlay.in_put.channel1
gpio_out = overlay.out_put.channel1

# 2. Input Data
pixels = [9, 1, 1, 1, 1, 1, 1, 1, 1]

kernel=[1,1,3,4,5,6,7,8,9]

print(f"--- BENCHMARK STARTING ---")
print(f"Data Size: {len(pixels)} Bytes")


print("\n[Race A] FPGA (via GPIO)...")
start_hw = time.perf_counter()

for p in pixels:
    data_only = p & 0xFF
    data_valid = (1 << 8) | p
    
    gpio_in.write(data_only, 0x1FF) 
    gpio_in.write(data_valid, 0x1FF) 
    gpio_in.write(data_only, 0x1FF) 

val = gpio_out.read()
hw_result = val & 0xFF

end_hw = time.perf_counter()
time_hw = end_hw - start_hw

print(f"FPGA Time:   {time_hw * 1000:.4f} ms")
print(f"FPGA Result: {hw_result}")

print("\n[Race B] CPU (Python Software)...")
start_sw = time.perf_counter()

s = 0
i = 0
for a in pixels:
    s += a * kernel[i]
    i += 1

if s > 255: s = 255
elif s < 0: s = 0
sw_result = s

end_sw = time.perf_counter()
time_sw = end_sw - start_sw

print(f"CPU Time:    {time_sw * 1000:.4f} ms")
print(f"CPU Result:  {sw_result}")
print("\n--- RESULTS ---")
if hw_result == sw_result:
    print("Accuracy: PASS (Results Match)")
else:
    print("Accuracy: FAIL (Results Differ)")

print(f"Speed Difference: Python is {time_hw / time_sw:.2f}x FASTER than FPGA (GPIO).")

--- BENCHMARK STARTING ---
Data Size: 9 Bytes

[Race A] FPGA (via GPIO)...
FPGA Time:   2.9305 ms
FPGA Result: 52

[Race B] CPU (Python Software)...
CPU Time:    1.9758 ms
CPU Result:  52

--- RESULTS ---
Accuracy: PASS (Results Match)
Speed Difference: Python is 1.48x FASTER than FPGA (GPIO).
